In [51]:
import tensorflow_datasets as tfds
import tensorflow as tf
import os
import csv
import pandas as pd
import numpy as np
from pyts.image import GramianAngularField
from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe
import keras
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import warnings
from keras.utils import to_categorical

In [2]:
#transfer signals into 2D images
path = "D:\\APE\\ml_model\\train_data\\" 
files= os.listdir(path) 
train_label = [] 
train_signal = [] #700 seq
length = []
gasf = GramianAngularField(image_size=28, method='summation')
for file in files: 
    data = np.loadtxt(open(path+file),delimiter=",",skiprows=0)
    seq_len = len(data) - 1
    length.append(seq_len)
    train_label.append(data[-1])
    X = data[0:seq_len]
    X = X.reshape(1, -1)
    X_gasf = gasf.fit_transform(X)
    train_signal.append(X_gasf[0])


train_sig = np.array(train_signal)
train_lab = np.array(train_label)
print(train_sig.shape)

path = "D:\\APE\\ml_model\\test_data\\" 
files= os.listdir(path) 
test_label = []
test_signal = [] #300 seq
length = []
for file in files: 
    data = np.loadtxt(open(path+file),delimiter=",",skiprows=0)
    seq_len = len(data) - 1
    length.append(seq_len)
    test_label.append(data[-1])
    X = data[0:seq_len]
    X = X.reshape(1, -1)
    X_gasf = gasf.fit_transform(X)
    test_signal.append(X_gasf[0])
    
test_sig = np.array(test_signal)
test_lab = np.array(test_label)
print(test_sig.shape)

E:\program files\Anaconda3\lib\site-packages\numba\np\ufunc\parallel.py:355: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 11004. The TBB threading layer is disabled.
  warnings.warn(problem)


(700, 28, 28)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
(300, 28, 28)


In [3]:
#RNN LSTM
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(500)),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dense(1)
])
    
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

history = model.fit(train_sig,train_lab, epochs=10,
                    validation_data=(test_sig,test_lab))

test_loss, test_acc = model.evaluate(test_sig,test_lab)

print(test_loss,test_acc)

Train on 700 samples, validate on 300 samples
Epoch 1/10
700/700 [==============================] - ETA: 58s - loss: 0.6883 - accuracy: 0.406 - ETA: 29s - loss: 0.9112 - accuracy: 0.453 - ETA: 19s - loss: 0.8376 - accuracy: 0.468 - ETA: 14s - loss: 0.7826 - accuracy: 0.523 - ETA: 11s - loss: 0.7777 - accuracy: 0.537 - ETA: 9s - loss: 0.7823 - accuracy: 0.520 - ETA: 7s - loss: 0.7692 - accuracy: 0.50 - ETA: 6s - loss: 0.7701 - accuracy: 0.49 - ETA: 5s - loss: 0.7654 - accuracy: 0.49 - ETA: 4s - loss: 0.7612 - accuracy: 0.50 - ETA: 4s - loss: 0.7541 - accuracy: 0.49 - ETA: 3s - loss: 0.7492 - accuracy: 0.49 - ETA: 3s - loss: 0.7426 - accuracy: 0.50 - ETA: 2s - loss: 0.7395 - accuracy: 0.50 - ETA: 2s - loss: 0.7340 - accuracy: 0.51 - ETA: 1s - loss: 0.7314 - accuracy: 0.51 - ETA: 1s - loss: 0.7295 - accuracy: 0.51 - ETA: 1s - loss: 0.7290 - accuracy: 0.50 - ETA: 0s - loss: 0.7263 - accuracy: 0.50 - ETA: 0s - loss: 0.7245 - accuracy: 0.50 - ETA: 0s - loss: 0.7263 - accuracy: 0.50 - 7s 10ms

In [67]:
#change length of signals into the longest one, many zeros in the tail of signals
train = pd.read_csv("D:\\APE\\ml_model\\train_data.csv",header=None)
train_label = to_categorical(np.array(train.iloc[:,0]))
train_signal = np.array(train.iloc[:,1:])
train_signal = train_signal.reshape(700,1,697)

test = pd.read_csv("D:\\APE\\ml_model\\test_data.csv",header=None)
test_label = to_categorical(np.array(test.iloc[:,0]))
test_signal = np.array(test.iloc[:,1:])
test_signal = test_signal.reshape(300,1,697)


print(train_signal.shape)
print(test_signal.shape)

#1D CNN
n_timesteps, n_features, n_outputs = train_signal.shape[1], train_signal.shape[2], train_label.shape[1]


model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters=1, kernel_size=1, activation='tanh', input_shape=(n_timesteps,n_features)))
model.add(tf.keras.layers.Conv1D(filters=1, kernel_size=1, activation='tanh'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.MaxPooling1D(pool_size=1))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100, activation='tanh'))
model.add(tf.keras.layers.Dense(n_outputs,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_signal, train_label, epochs=10)
_, accuracy = model.evaluate(test_signal, test_label, verbose=0)
print(accuracy)

(700, 1, 697)
(300, 1, 697)
Train on 700 samples
Epoch 1/10
700/700 [==============================] - ETA: 8s - loss: 0.7004 - accuracy: 0.46 - 0s 665us/sample - loss: 0.6952 - accuracy: 0.4857
Epoch 2/10
700/700 [==============================] - ETA: 0s - loss: 0.6942 - accuracy: 0.50 - 0s 63us/sample - loss: 0.6934 - accuracy: 0.5171
Epoch 3/10
700/700 [==============================] - ETA: 0s - loss: 0.6661 - accuracy: 0.62 - 0s 63us/sample - loss: 0.6972 - accuracy: 0.4900
Epoch 4/10
700/700 [==============================] - ETA: 0s - loss: 0.7089 - accuracy: 0.40 - 0s 63us/sample - loss: 0.6954 - accuracy: 0.4843
Epoch 5/10
700/700 [==============================] - ETA: 0s - loss: 0.6924 - accuracy: 0.53 - 0s 76us/sample - loss: 0.6944 - accuracy: 0.4914
Epoch 6/10
700/700 [==============================] - ETA: 0s - loss: 0.6926 - accuracy: 0.50 - 0s 64us/sample - loss: 0.6939 - accuracy: 0.4886
Epoch 7/10
700/700 [==============================] - ETA: 0s - loss: 0.6912 - a